In [1]:
import nibabel as nib
import numpy as np
from nilearn import datasets,maskers, plotting,image,connectome
from bids.layout import BIDSLayout

In [2]:
#base directory
fmriprep_dir='../Full-Dataset'
layout=BIDSLayout(fmriprep_dir)

In [3]:
len(layout.get_subjects())

71

In [4]:
# Fetch the AAL atlas
aal_atlas = datasets.fetch_atlas_aal('SPM12')

# Load the AAL atlas image
aal_img = nib.load(aal_atlas.maps)

In [5]:
aal_img.shape

(91, 109, 91)

In [8]:
aal_atlas.indices

['2001',
 '2002',
 '2101',
 '2102',
 '2111',
 '2112',
 '2201',
 '2202',
 '2211',
 '2212',
 '2301',
 '2302',
 '2311',
 '2312',
 '2321',
 '2322',
 '2331',
 '2332',
 '2401',
 '2402',
 '2501',
 '2502',
 '2601',
 '2602',
 '2611',
 '2612',
 '2701',
 '2702',
 '3001',
 '3002',
 '4001',
 '4002',
 '4011',
 '4012',
 '4021',
 '4022',
 '4101',
 '4102',
 '4111',
 '4112',
 '4201',
 '4202',
 '5001',
 '5002',
 '5011',
 '5012',
 '5021',
 '5022',
 '5101',
 '5102',
 '5201',
 '5202',
 '5301',
 '5302',
 '5401',
 '5402',
 '6001',
 '6002',
 '6101',
 '6102',
 '6201',
 '6202',
 '6211',
 '6212',
 '6221',
 '6222',
 '6301',
 '6302',
 '6401',
 '6402',
 '7001',
 '7002',
 '7011',
 '7012',
 '7021',
 '7022',
 '7101',
 '7102',
 '8101',
 '8102',
 '8111',
 '8112',
 '8121',
 '8122',
 '8201',
 '8202',
 '8211',
 '8212',
 '8301',
 '8302',
 '9001',
 '9002',
 '9011',
 '9012',
 '9021',
 '9022',
 '9031',
 '9032',
 '9041',
 '9042',
 '9051',
 '9052',
 '9061',
 '9062',
 '9071',
 '9072',
 '9081',
 '9082',
 '9100',
 '9110',
 '9120',
 

In [11]:
# Assuming you have an fMRI image 'fmri_img', load it using nibabel
fmri_img = nib.load(r'C:\Users\reub\Downloads\exmni_filtered_func_data.nii.gz')

# Extract data from the fMRI image
fmri_data = fmri_img.get_fdata()
fmri_data.shape

(91, 109, 91, 336)

In [10]:
# Create an array to store the ROI data
roi_data = np.zeros((len(aal_atlas.labels), fmri_data.shape[-1]))

In [28]:
roi_mask = (aal_img.get_fdata() == 2001)
roi_mask
roi_data=fmri_data[roi_mask]
len(roi_data)

3526

In [12]:
# # Iterate through each region in the AAL atlas
# for i, label in enumerate(aal_atlas.labels):
#     # Get the region ID corresponding to the label
#     region_id = int(aal_atlas.indices[i])

#     # Mask the fMRI data with the region's binary mask from the atlas
#     roi_mask = (aal_img.get_fdata() == region_id)
#     roi_data[i, :] = np.mean(fmri_data[roi_mask], axis=0)

# # Now, 'roi_data' contains the mean time series for each ROI

# # Optionally, you can also obtain the coordinates of each ROI
# roi_coordinates = np.array([np.where(aal_img_resampled.get_fdata() == int(aal_atlas.indices[i])) for i in range(len(aal_atlas.labels))])

In [13]:
n=0
# Iterate through each region in the AAL atlas
for i, label in enumerate(aal_atlas.labels):
    # Get the region ID corresponding to the label
    region_id = int(aal_atlas.indices[i])

    # Mask the fMRI data with the region's binary mask from the atlas
    roi_mask = (aal_img.get_fdata() == region_id)
    roi_data = fmri_data[roi_mask]
    if roi_data.shape[0]<=100:
        n+=1
        print(label)
print(n)

# Now, 'roi_data' contains the mean time series for each ROI

# Optionally, you can also obtain the coordinates of each ROI
# roi_coordinates = np.array([np.where(aal_img_resampled.get_fdata() == int(aal_atlas.indices[i])) for i in range(len(aal_atlas.labels))])

Vermis_1_2
1


In [29]:
def extract_mean_time_series(roi_data,s):
    # Extract mean time series from a random one-third of voxels in the ROI
    
    num_voxels = roi_data.shape[0]
    num_selected_voxels = num_voxels // 3
    # Perform bootstrapping by sampling with replacement
    np.random.seed(s)
    selected_voxels_index = np.random.choice(len(roi_data), size=num_selected_voxels,replace=False)
    extracted_data = np.array([roi_data[idx] for idx in selected_voxels_index])
    mean_time_series = np.mean(extracted_data, axis=0)
    return mean_time_series

In [30]:
def data_augmentation(fmri_files,no_of_augmentations):
    all_sub_aug_t=[]
    all_sub_aug_corr=[]
    for f in fmri_files:
        # Assuming you have an fMRI image 'fmri_img', load it using nibabel
        fmri_img = nib.load(f)
        # Extract data from the fMRI image
        fmri_data = fmri_img.get_fdata()
        for s in range(no_of_augmentations):
            augmented_time_sample=[]
            # Iterate through each region in the AAL atlas
            for i, label in enumerate(aal_atlas.labels):
                # Get the region ID corresponding to the label
                region_id = int(aal_atlas.indices[i])
                # Mask the fMRI data with the region's binary mask from the atlas
                roi_mask = (aal_img.get_fdata() == region_id)
                roi_data = fmri_data[roi_mask]
                mean_t_s=extract_mean_time_series(roi_data,s)
                augmented_time_sample.append(mean_t_s)
            sub_aug_ts=np.array(augmented_time_sample)
            sub_aug_corr=np.corrcoef(sub_aug_ts)
            all_sub_aug_t.append(sub_aug_ts)
            all_sub_aug_corr.append(sub_aug_corr)
    all_sub_aug_corr=np.array(all_sub_aug_corr)
    all_sub_aug_t=np.array(all_sub_aug_t)
    return all_sub_aug_corr,all_sub_aug_t

        


In [31]:
fmri_files=[r'C:\Users\reub\Downloads\exmni_filtered_func_data.nii.gz']
all_sub_aug_corr,all_sub_aug_t=data_augmentation(fmri_files,10)

In [33]:
all_sub_aug_corr.shape
all_sub_aug_t.shape

(10, 116, 336)

In [39]:
plotting.plot_matrix(all_sub_aug_corr[1], figure=(10, 8), labels=aal_atlas.labels,
                     vmax=0.8, vmin=-0.8);

In [43]:
(all_sub_aug_corr[0],all_sub_aug_corr[1])

(array([[ 1.        ,  0.83581652,  0.60119122, ..., -0.1923339 ,
         -0.02708919, -0.03008906],
        [ 0.83581652,  1.        ,  0.55791746, ..., -0.15918404,
         -0.08866728, -0.05932157],
        [ 0.60119122,  0.55791746,  1.        , ..., -0.0035176 ,
          0.03961542, -0.06902491],
        ...,
        [-0.1923339 , -0.15918404, -0.0035176 , ...,  1.        ,
          0.37167403,  0.39584145],
        [-0.02708919, -0.08866728,  0.03961542, ...,  0.37167403,
          1.        ,  0.62880982],
        [-0.03008906, -0.05932157, -0.06902491, ...,  0.39584145,
          0.62880982,  1.        ]]),
 array([[ 1.        ,  0.83101096,  0.56397837, ..., -0.20862362,
         -0.04473297, -0.03600071],
        [ 0.83101096,  1.        ,  0.52590582, ..., -0.16007769,
         -0.0915525 , -0.0464739 ],
        [ 0.56397837,  0.52590582,  1.        , ...,  0.0130944 ,
          0.05211237, -0.05403463],
        ...,
        [-0.20862362, -0.16007769,  0.0130944 , ...,  